In [205]:
subcases_pattern="1_1715"
reference_location="../.."
source_name="Her X-1"
osa_version='OSA11.1' #-dev210827.0528-37487--iisglobal'
#osa_version='OSA11.1-dev210827.0528-37487--iisglobal'
nscw=-1
mosaic=0
chi2_limit=1.2
systematic_fraction=0.01
search_time=-1 #search time of INTEGRAL DATA around the time of observation (days), if -1 uses the actual interval
integral_data_rights="all-private"

In [119]:
if reference_location == 'None':
    import subprocess
    logfile_name = 'log_download_reference.txt'
    logfile = open(logfile_name, 'w')
    cmd = 'wget https://www.isdc.unige.ch/~ferrigno/Downloads/subcases_herX1_nustar.tgz;tar xfz subcases_herX1_nustar.tgz;rm subcases_herX1_nustar.tgz'
    # cmd='pwd;ls'
    out = subprocess.call(cmd, stdout=logfile, stderr=logfile, shell=True)
    logfile.close()
    if out != 0:
        print('Exit status of download and decompression is %d')
        raise RuntimeError
    
    reference_location='.'

#  ISGRI verification with NuSTAR reference

simultaneous observations allow easy comparisong

normalization need to be fitted and reported

In [120]:
%matplotlib notebook
#import matplotlib.pylab as plt


import numpy as np
import importlib
from astroquery.simbad import Simbad
from astropy import coordinates as coord
import os
import numpy as np
import glob
import json

from astropy.io import fits as pf
from astropy.time import Time
from astropy.time import TimeDelta

from astropy import table



In [121]:

import requests
url="https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw/timesystem/api/v1.0/scwlist/cons/"
def get_scw_list(subcase_dir, ra,dec,time, delta_time): #time in UTC T (2019-06-23T16:53:39.000), deltat in days
    
    try:
        
        scwlist = open(subcase_dir+"/ISGRI_scw_list.txt").read().split()
        print("read scw list from " + subcase_dir+"/ISGRI_scw_list.txt")
        return scwlist
    except:


    #     result_table = Simbad.query_object(source_name)
    #     source_coord = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], unit=("hourangle", "deg"))

    #     ra=source_coord.ra.deg
    #     dec=source_coord.dec.deg

        dt = TimeDelta(delta_time)

        Tstart = Time(time) - dt
        Tstop  = Time(time) + dt

        Tstart.format='isot'
        Tstop.format='isot'


        params=Tstart.value+'/'+Tstop.value+'?&ra='+str(ra)+'&dec='+str(dec)+'&radius='+str(6.0)+'&min_good_isgri=1000'
        print("Obtained scw list online from " + url+params)
        return requests.get(url+params).json() #,cookies=cookies).json()



In [122]:
reference_location

# if reference_location  not git or http may be a problem
# dependency injection for this, if can

'../..'

In [123]:

subcase=None
reference_spectra = []
reference_backs = []
reference_rmfs = []
reference_arfs = []
fns = []

all_selected_subcases = glob.glob(reference_location+"/subcases/*"+subcases_pattern+"*")

for subcase_dir in all_selected_subcases:
    print("inspecting", subcase_dir)
    
    fns = [os.path.basename(fn) for fn in sorted(glob.glob(subcase_dir+"/*"))]
    
    try:
        #If it is rebinned with my script
        reference_spectra = [fn for fn in fns if fn.endswith("rbn.pi")]
        
        reference_rmfs = [fn for fn in fns if fn.endswith("sr.rmf") or fn.endswith("sr.rmf.gz")]
        
        reference_arfs = [fn for fn in fns if fn.endswith("sr.arf")]
        
        reference_backs = [fn for fn in fns if fn.endswith("bk.pha")]
        
    except: 
        print("Not found Nustar Files in " + subcase_dir)
        continue
        
    
    try:
        print(subcase_dir+'/'+reference_spectra[0])
        spec=pf.open(subcase_dir+'/'+reference_spectra[0])

        t1=Time(spec[1].header['DATE-OBS'])
        t2=Time(spec[1].header['DATE-END'])

        ra=spec[1].header['RA_OBJ']
        dec=spec[1].header['DEC_OBJ']

        delta_time = (t2 - t1 )/2

        time=t1+delta_time

        spec.close()

        if search_time == -1:
            scwlist= get_scw_list(subcase_dir, ra,dec, time, delta_time)
        else:
            scwlist= get_scw_list(subcase_dir, ra,dec, time, search_time)
        
        print(t1, time)
        
        #print(scwlist)
    except:
        print("Not getting times")
        continue
    
   
        
    #print(fns)
    
    reference_dir=subcase_dir
    
    #print("found",reference_dir, reference_spectra, len(scwlist),len(ref_sources), ref_sources)
        
    break
        
if len(list(zip(reference_spectra, reference_backs, reference_rmfs, reference_arfs))) == 0:
    raise Exception(f"no compatible NuSTAR spectra found; last fns tried {fns} all selected subcases {all_selected_subcases} in {reference_location}")
#reference_spectra, reference_rmfs


inspecting ../../subcases/Her_X-1_1715
Not getting times
inspecting ../../subcases/Her_X-1_1715_nustar
../../subcases/Her_X-1_1715_nustar/nu10202002002A01_sr_rbn.pi
Obtained scw list online from https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw/timesystem/api/v1.0/scwlist/cons/2016-08-20T04:31:08.000/2016-08-21T06:21:08.000?&ra=254.45754&dec=35.34233&radius=6.0&min_good_isgri=1000
2016-08-20T04:31:08.000 2016-08-20T17:26:08.000


In [124]:
print("searching for coordinates of", source_name)

result_table = Simbad.query_object(source_name)
source_coord = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], unit=("hourangle", "deg"))

result_table

searching for coordinates of Her X-1


MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,int32
V* HZ Her,16 57 49.8110,+35 20 32.487,14,14,0.022,0.024,90,A,O,2018yCat.1345....0G,1


In [168]:
import random
if nscw >0:
    random.seed(0)
    cleaned_list = [
                s+"."+"001"
                for s in list(sorted(set( scwlist  ))) 
                if s.endswith("0010")
            ]
    if len(cleaned_list) > nscw:
        scw_pick = random.sample(cleaned_list,nscw)
    else:
        print("nscw (%d) > than available scw (%d), using them all"%(nscw,len(cleaned_list)))
        scw_pick = cleaned_list


    scw_list_str = ",".join(sorted(scw_pick))
else:
    scw_list_str=",".join([s+"."+"001" for s in sorted(set( sorted(scwlist)  ))])

#print(len(scw_list_str))

scw_list_str

'171500480010.001,171500490010.001,171500500010.001,171500510010.001,171500520010.001,171500530010.001,171500540010.001,171500550010.001,171500560010.001,171500570010.001,171500580010.001,171500600010.001,171500610010.001,171500620010.001,171500650010.001,171500660010.001,171500670010.001,171500680010.001,171500690010.001,171500700010.001,171500710010.001,171500720010.001,171500730010.001,171500740010.001,171500750010.001,171500760010.001,171500770010.001,171500780010.001,171500790010.001,171500800010.001,171500810010.001,171500820010.001,171500830010.001,171500850010.001,171500860010.001,171500870010.001,171500880010.001'

In [212]:
import integralclient as ic
scw_list_str = ",".join(scw_list_str.split(",")[:2])

ic.get_sc(ic.converttime("SCWID", scw_list_str.split(",")[0][:-4], "ANY")['UTC'].split()[0],
          ra=ra,
          dec=dec)

scw_list_str.split(",")[0][:-4] 

2021-09-28 19:11:59 87be31766f34 root[468] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/SCWID/171500480010/ANY


'171500480010'

In [173]:
import oda_api.api
import importlib
importlib.reload(oda_api)

# osa versions with '-' use ic root version, only available on staging-1-3
print('will find appropriate API for OSA version', osa_version)
if '-' in osa_version:
    print('osa version has subversion - will use mmoda')
    disp = oda_api.api.DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data/")
else:
    try:
        disp = oda_api.api.DispatcherAPI(url="http://dispatcher.staging.internal.odahub.io")
        disp.get_instrument_description("isgri")    
    except Exception as e:
        print('\033[31mFAILED to access staging-1-2, will proceed to query production: note that it can not access private data!\033[0m')
        print('\033[31mexception was: ', e,'\033[0m')
        try:
            disp = oda_api.api.DispatcherAPI(url='https://www.astro.unige.ch/cdci/astrooda/dispatch-data')
            disp.get_instrument_description("isgri")
        except:
            raise ConnectionError


will find appropriate API for OSA version OSA11.0-dev210827.0528-37487--iisglobal
osa version has subversion - will use mmoda


In [179]:
api_cat = None

api_file_list =  glob.glob(subcase_dir+"/api_cat_str_*.txt")

if len(api_file_list) == 0:
    pass
elif len(api_file_list) > 1:
    raise RuntimeError(f'should not be many of these: {api_file_list}')
else:
    print('Found api catalog in file ', api_file_list[0])
    with open(api_file_list[0]) as ff:
        api_cat_struct = json.load(ff)
        api_cat = json.dumps(api_cat_struct)


Found api catalog in file  ../../subcases/Her_X-1_1715_nustar/api_cat_str_Her_X-1.txt


In [180]:
api_cat

'{"cat_frame": "fk5", "cat_coord_units": "deg", "cat_column_list": [[3, 36, 44, 45, 210, 0], ["1E 1740.7-2942", "GRO J1719-24", "GRS 1758-258", "GX 1+4", "KS 1741-293", "Her X-1"], [14.02076244354248, 113.27212524414062, 7.533890247344971, 44.93584442138672, 9.441457748413086, 0.0], [265.97607421875, 259.9058837890625, 270.2437744140625, 263.0091552734375, 266.2197570800781, 254.45754], [-29.75826644897461, -25.02088165283203, -25.748001098632812, -24.742778778076172, -29.39611053466797, 35.34233], [-32768, -32768, -32768, -32768, -32768, 0], [2, 2, 2, 2, 1, 2], [0, 0, 0, 0, 0, 0], [2.9999999242136255e-05, 0.00014000000373926014, 0.0002800000074785203, 0.0002800000074785203, 0.009999999776482582, 0.0]], "cat_column_names": ["meta_ID", "src_names", "significance", "ra", "dec", "NEW_SOURCE", "ISGRI_FLAG", "FLAG", "ERR_RAD"], "cat_column_descr": [["meta_ID", "<i8"], ["src_names", "<U14"], ["significance", "<f8"], ["ra", "<f8"], ["dec", "<f8"], ["NEW_SOURCE", "<i8"], ["ISGRI_FLAG", "<i8"],

In [182]:
def ensure_source_in_api_cat(_api_cat):
    if _api_cat is None:
        return _api_cat
        
    api_cat_dp = oda_api.data_products.ApiCatalog(json.loads(_api_cat))

    m_source = api_cat_dp.table['src_names'] == source_name
    if api_cat_dp.table['ISGRI_FLAG'][m_source] != [1]:
        print(f"WARNING: ISGRI_FLAG was not set for source of interest ({source_name}), setting to 1")

        api_cat_dp.table['ISGRI_FLAG'][m_source] = 1
        _api_cat = api_cat_dp.get_api_dictionary()
    return _api_cat

api_cat = ensure_source_in_api_cat(api_cat)

In [213]:
if api_cat is None:

    for c_emin in [28]:

        image = disp.get_product(instrument="isgri", 
                         product="isgri_image", 
                         product_type="Real", 
                         osa_version=osa_version,
                         E1_keV=float(np.round(c_emin)),
                         E2_keV=80.0,
                         integral_data_rights=integral_data_rights,
                         scw_list=scw_list_str,
                         token=oda_api.token.discover_token())

        sources=image.dispatcher_catalog_1.table[image.dispatcher_catalog_1.table['significance']>=6.0]
        #source = sources[sources['src_names']==source_name]
        unique_sources=table.unique(sources, keys=['src_names'])
else:
    print("api_cat exists!")
        
        
        #print(source)

#         d=image.mosaic_image_0.data_unit[1].data


#         img=np.array(d.data)

#         m_bkg=img<10
#         m_bkg&=img!=0

#         img_std = np.std(img[m_bkg])

#         img[np.array(img)>img_std*5]=img_std*5

        #plt.imshow(img)

#         by_lt['%.10lg'%c_emin]=dict(
#             emin=c_emin,            
#             imgstd=img_std,
#         )
#         by_lt['%.10lg'%c_emin].update(dict([(n, source[n]) for n in source.colnames]))

        
#     for ltb, c in by_lt.items():
#         print(lt, c['significance'],c['significance']/c['imgstd'])

api_cat exists!


In [184]:
##Removes new sources and adds our if not found
if api_cat is None:
    FLAG=0
    torm=[]
    for ID,n in enumerate(unique_sources['src_names']):
        if(n[0:3]=='NEW'):
            torm.append(ID)
        if(n==source_name):
            FLAG=1

    unique_sources.remove_rows(torm)
    nrows=len(unique_sources['src_names'])

    if FLAG==0:
        unique_sources.add_row((0,source_name,0,ra,dec,0,2,0,0))

    image.dispatcher_catalog_1.table = unique_sources

    api_cat=image.dispatcher_catalog_1.get_api_dictionary()
    
    with open(subcase_dir+"/api_cat_str_%s.txt"%(source_name.replace(' ','_').replace('+','p')),'w') as f: 
        f.write(api_cat) 



In [185]:
oda_api.data_products.ApiCatalog(json.loads(api_cat)).table

meta_ID,src_names,significance,ra,dec,NEW_SOURCE,ISGRI_FLAG,FLAG,ERR_RAD
,,,deg,deg,,,,
int64,str14,float64,float64,float64,int64,int64,int64,float64
3,1E 1740.7-2942,14.02076244354248,265.97607421875,-29.75826644897461,-32768,2,0,2.9999999242136255e-05
36,GRO J1719-24,113.27212524414062,259.9058837890625,-25.02088165283203,-32768,2,0,0.00014000000373926014
44,GRS 1758-258,7.533890247344971,270.2437744140625,-25.748001098632812,-32768,2,0,0.0002800000074785203
45,GX 1+4,44.93584442138672,263.0091552734375,-24.742778778076172,-32768,2,0,0.0002800000074785203
210,KS 1741-293,9.441457748413086,266.2197570800781,-29.39611053466797,-32768,1,0,0.009999999776482582
0,Her X-1,0.0,254.45754,35.34233,0,1,0,0.0


In [215]:
scw_list_str

'171500480010.001'

In [222]:
spectrum = disp.get_product(instrument="isgri", 
                 product="isgri_spectrum", 
                 product_type="Real", 
                 osa_version=osa_version,
                 E1_keV=35.0,
                 E2_keV=80.0,
                 integral_data_rights=integral_data_rights,
                 scw_list=scw_list_str,
                 token=oda_api.token.discover_token(),
                 selected_catalog=api_cat,
                 max_pointings=50)

{l.meta_data['src_name']:l for l in spectrum._p_list if l.meta_data['src_name']}

2021-09-28 20:03:15 87be31766f34 oda_api.token[468] INFO decoding token with jwt and NOT verifying
2021-09-28 20:03:15 87be31766f34 oda_api.token[468] INFO found token in environment variable ODA_TOKEN your token payload: {
    "email": "Volodymyr.Savchenko@unige.ch",
    "exp": 1634486483.5381484,
    "msdone": false,
    "mssub": false,
    "name": "sitamin",
    "roles": "authenticated user, administrator, content manager, general, integral-private-qla, magic, unige-hpc-full, public-pool-hpc, antares",
    "sub": "Volodymyr.Savchenko@unige.ch"
}
2021-09-28 20:03:15 87be31766f34 oda_api.token[468] INFO token expires in 452.0 h
2021-09-28 20:03:15 87be31766f34 oda_api.api.dispatcherapi.progress[468] INFO - waiting for remote response (since 2021-09-28 20:03:15), please wait for https://www.astro.unige.ch/mmoda/dispatch-data//run_analysis
2021-09-28 20:03:40 87be31766f34 oda_api.api[468] INFO session: D9AFZA5VPUEMOPJ8 job: 81ddf178197c78eb
2021-09-28 20:03:40 87be31766f34 oda_api.api.d

{'Her X-1': <oda_api.data_products.NumpyDataProduct at 0x7fd5702b1d60>,
 'Background': <oda_api.data_products.NumpyDataProduct at 0x7fd58655e0a0>}

In [223]:
specprod=[l for l in spectrum._p_list if l.meta_data['src_name'] == source_name]    

spec_fn="/tmp/isgri_spectrum_{}.fits".format(source_name.replace(' ', '_').replace('+','p'))
arf_fn="/tmp/isgri_arf_{}.fits".format(source_name.replace(' ', '_').replace('+','p'))
rmf_fn="/tmp/isgri_rmf_{}.fits".format(source_name.replace(' ', '_').replace('+','p'))

specprod[0].write_fits_file(spec_fn)
specprod[1].write_fits_file(arf_fn)
specprod[2].write_fits_file(rmf_fn)

In [224]:
from astropy.io import fits

for s, b, r, a in zip(reference_spectra, reference_backs, reference_rmfs, reference_arfs):

    fs=fits.open(reference_dir+"/"+s)
    fr=fits.open(reference_dir+"/"+r)
    fb=fits.open(reference_dir+"/"+b)
    fa=fits.open(reference_dir+"/"+a)
    

    fs[2].header['RESPFILE'] = 'NONE'
    fs[2].header['ANCRFILE'] = 'NONE'
    fs[2].header['BACKFILE'] = 'NONE'
    
    #f[2].header['RESPFILE']
    if "A" in s:
        fn = "reference_spec_A.fits"
    else:
        fn = "reference_spec_B.fits"
        
    print("writing", fn)        
    fs.writeto(fn, overwrite=True)
    
    if 'A' in r:
        fn = "reference_rmf_A.fits"
    else:
        fn = "reference_rmf_B.fits"
        
    print("writing", fn)        
    fr.writeto(fn, overwrite=True)
    
    if 'A' in b:
        fn = "reference_bkg_A.fits"
    else:
        fn = "reference_bkg_B.fits"

    print("writing", fn)        
    fb.writeto(fn, overwrite=True)
    
    if 'A' in a:
        fn = "reference_arf_A.fits"
    else:
        fn = "reference_arf_B.fits"
        
    print("writing")
    fa.writeto(fn, overwrite=True)
    
    fs.close()
    fb.close()
    fa.close()
    fr.close()


writing reference_spec_A.fits
writing reference_rmf_A.fits
writing reference_bkg_A.fits
writing
writing reference_spec_B.fits
writing reference_rmf_B.fits
writing reference_bkg_B.fits
writing


In [ ]:
reference_spec_A="reference_spec_A.fits"
reference_spec_B="reference_spec_B.fits"
reference_arf_A="reference_arf_A.fits"
reference_arf_B="reference_arf_B.fits"
reference_rmf_A="reference_rmf_A.fits"
reference_rmf_B="reference_rmf_B.fits"
reference_bkg_A="reference_bkg_A.fits"
reference_bkg_B="reference_bkg_B.fits"
isgri_spec=spec_fn
isgri_arf=arf_fn
isgri_rmf=rmf_fn